In [ ]:
import pandas as pd
from IPython.display import display

# CSV inlezen
df = pd.read_csv("df.csv")

# Filter alleen relevante kolommen
df_filtered = df[["dossier_id", "dossier_name", "file_name", "dossier_published_date", "dossier_decision_date"]]

# Verwijder metadata.json en groepeer per dossier_id
df_aggregated = (
    df_filtered[df_filtered["file_name"] != "metadata.json"]
    .groupby(["dossier_id", "dossier_name", "dossier_published_date", "dossier_decision_date"])
    .agg(document_count=("file_name", "count"))  # Tel documenten per dossier
    .reset_index()
)

# Instellingen om lange tekst volledig te tonen
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)

# DataFrame weergeven in Jupyter Notebook
display(df_aggregated)

# Opslaan als CSV
df_aggregated.to_csv("df_dossiers_subset.csv", index=False)

print("✅ Geaggregeerde CSV opgeslagen als 'df_dossiers_subset.csv'")


In [ ]:
import pandas as pd
from IPython.display import display

# CSV inlezen
df = pd.read_csv("df.csv")

# Verwijder spaties in kolomnamen (indien nodig)
df.columns = df.columns.str.strip()

# Controleer of de verwachte kolommen aanwezig zijn
expected_columns = ["dossier_id", "file_name", "file_type"]
missing_columns = [col for col in expected_columns if col not in df.columns]

if missing_columns:
    print(f"❌ De volgende kolommen ontbreken in de CSV: {missing_columns}")
else:
    # Selecteer relevante kolommen en hernoem file_name naar document_title
    df_documents = df[["dossier_id", "file_name", "file_type"]].rename(
        columns={"file_name": "document_title", "file_type": "document_type"}
    )

    # Verwijder metadata.json
    df_documents = df_documents[df_documents["document_title"] != "metadata.json"]

    # Verwijder de punt uit document_type (extensie)
    df_documents["document_type"] = df_documents["document_type"].str.replace(".", "", regex=False)

    # Document-ID toevoegen (per dossier_id een oplopende teller)
    df_documents["document_id"] = (
        df_documents.groupby("dossier_id").cumcount() + 1  # Start teller bij 1 per dossier
    ).astype(str)

    # Combineer document_id met dossier_id
    df_documents["document_id"] = df_documents["dossier_id"].astype(str) + "-" + df_documents["document_id"]

    # Kolommen opnieuw ordenen
    df_documents = df_documents[["dossier_id", "document_id", "document_title", "document_type"]]

    # Opslaan als nieuwe CSV
    df_documents.to_csv("df_documents.csv", index=False)

    # DataFrame weergeven
    display(df_documents)
    print("✅ Geëxporteerd als 'df_documents.csv'")


In [ ]:
import time

def scrape_document_metadata(dossier_url):
    """Scrapet documentmetadata van alle pagina's binnen een dossier met een crawl delay."""

    # Extract dossier ID uit de URL
    dossier_id_match = re.search(r'/dossier/([^/]+)', dossier_url)
    dossier_id_str = dossier_id_match.group(1) if dossier_id_match else None

    metadata_list = []
    seen_documents = set()
    page = 1
    max_pages = 80 # Failsafe om te voorkomen dat hij oneindig blijft lopen

    while page <= max_pages:
        paged_url = f"{dossier_url}?prefix=VWS-WC&dossierId={dossier_id_str}&pu={page}#tabcontrol-1"
        response = requests.get(paged_url)
        if response.status_code != 200:
            print(f"Fout bij ophalen van {paged_url}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        table_rows = soup.select(".woo-table tbody tr")
        if not table_rows:
            print(f"Geen documenten gevonden op pagina {page}, stoppen met scrapen.")
            break

        new_documents_found = False

        for row in table_rows:
            try:
                # Document ID ophalen uit de eerste kolom
                doc_id_tag = row.select_one("td:nth-of-type(1)")
                doc_id = doc_id_tag.text.strip() if doc_id_tag else None

                # Document URL ophalen
                doc_link_tag = row.select_one("td:nth-of-type(3) a")
                doc_url = doc_link_tag["href"] if doc_link_tag else ""

                if not doc_id or doc_id in seen_documents:
                    continue

                seen_documents.add(doc_id)
                new_documents_found = True

                # Documenttype
                doc_type_tag = row.select_one("td:nth-of-type(2) use")
                doc_type_key = doc_type_tag["xlink:href"].split("#")[-1] if doc_type_tag else "unknown"
                doc_type = DOCUMENT_TYPE_MAP.get(doc_type_key, "Onbekend")

                # Documentdatum correct ophalen
                doc_date_tag = row.select_one("td:nth-of-type(4) time")
                doc_date = doc_date_tag["datetime"] if doc_date_tag else None

                metadata_list.append({
                    "document_id": doc_id,
                    "document_sourcetype": doc_type,
                    "document_date": doc_date,
                    "dossier_id": dossier_id
                })
            except Exception as e:
                print(f"Fout bij parsen: {e}")

        if not new_documents_found:
            print(f"Geen nieuwe documenten meer gevonden op pagina {page}, stoppen met scrapen.")
            break

        page += 1
        time.sleep(1)  # Crawl delay na elke pagina scrape

    for doc in metadata_list:
        print(doc)

    return metadata_list


In [ ]:
# scrape de datum van

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Mapping van documenttypes
DOCUMENT_TYPE_MAP = {
    "unknown": "Onbekend",
    "pdf": "PDF",
    "email": "E-mailbericht",
    "presentation": "Presentatie",
    "doc": "Word-document",
    "chat": "Chatbericht",
    "image": "Afbeelding",
    "spreadsheet": "Spreadsheet"
}

# Inlezen van de bestaande documentgegevens
df_documents = pd.read_csv("df_ALL_documents.csv", dtype={"document_id": str})
df_dossiers = pd.read_csv("../../dataset/ALL/df_dossiers.csv", dtype={"dossier_id": int})

# Handmatig dossier_id invoeren
dossier_id = 131

dossier = df_dossiers[df_dossiers["dossier_id"] == dossier_id].iloc[0]
dossier_url = dossier["dossier_sourceURL"]

print(f"Scrapen: {dossier_url}")
document_metadata = scrape_document_metadata(dossier_url)

# Omzetten naar dataframe
df_metadata = pd.DataFrame(document_metadata)

def extract_prefix(document_title):
    """Probeert het juiste dossierprefix te extraheren uit de document_title."""

    # Specifieke uitzondering: als het document begint met "VWS-WC-19.00-1-", dan is de prefix gewoon "VWS-WC"
    if document_title.startswith("CCMO-WOO-00-104.pdf"):
        return "CCMO-WOO-00"

    # Standaardpatronen voor prefixes
    match = re.match(r"^(VWS-WC-\d{3}|VWS-WOO-[A-Za-z0-9]+|VWS-WC)", document_title)
    return match.group(1) if match else None

def match_documents(df_documents, df_metadata, dossier_id):
    """Matcht gescrapete metadata met documenten op basis van dossierprefix en document_id."""
    df_documents_dossier = df_documents[df_documents["dossier_id"] == dossier_id]

    if df_documents_dossier.empty:
        print(f"⚠️ Geen documenten gevonden voor dossier_id {dossier_id}")
        return df_documents

    # Zoek het eerste document dat GEEN generieke naam heeft
    sample_document_title = None
    for title in df_documents_dossier["document_title"]:
        if not title.lower().startswith("besluitbrief"):  # Voeg andere generieke namen toe indien nodig
            sample_document_title = title
            break

    if not sample_document_title:
        print(f"❌ Geen geldig document gevonden om een prefix uit te halen voor dossier {dossier_id}.")
        return df_documents

    dossier_prefix = extract_prefix(sample_document_title)

    if not dossier_prefix:
        print(f"❌ Geen geldig dossierprefix gevonden in: {sample_document_title}")
        return df_documents

    print(f"🔎 Dossierprefix bepaald als: {dossier_prefix}")

    for i, doc_row in df_documents_dossier.iterrows():
        document_title = doc_row["document_title"]
        print(f"\n---\n🔍 Matching document: {document_title}")

        for _, meta_row in df_metadata.iterrows():
            meta_doc_id = str(meta_row["document_id"]).strip()  # Zorgt dat het ID een string is
            expected_identifier = f"{dossier_prefix}-{meta_doc_id}"  # Verwachte match

            # Exacte match met regex
            if re.search(rf"\b{re.escape(expected_identifier)}\b", document_title):
                print(f"  ✅ MATCH GEVONDEN! Updaten document {meta_doc_id}")

                df_documents.at[i, "document_sourcetype"] = meta_row["document_sourcetype"]
                df_documents.at[i, "document_date"] = meta_row["document_date"]

                # Print de waarde die in de CSV wordt gezet
                print(f"  📌 In CSV zetten → Type: {meta_row['document_sourcetype']}, Datum: {meta_row['document_date']}")
                break  # Stop met zoeken zodra de match is gevonden

    return df_documents

df_documents = match_documents(df_documents, df_metadata, dossier_id)

# Opslaan van de verrijkte dataset
df_documents.to_csv("df_ALL_documents.csv", index=False)
print(f"Scraping en verrijking voltooid voor dossier {dossier_id}!")
print(dossier_url)

In [3]:
# Scrape relaties van documenten en voeg ze toe als nieuwe kolommen

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Laad de CSV
df = pd.read_csv("../../dataset/VWS/VWS_documents.csv")

# Zoek de startindex voor document_id 210-1626
start_index = df[df["document_id"] == "210-1626"].index[0]

# Filter de dataset vanaf 205-3037
df = df.iloc[start_index:].copy()

# Voeg nieuwe kolommen toe met standaardwaarde "Geen"
df["document_attachments"] = "Geen"
df["document_same_emailconvo"] = "Geen"
df["document_related"] = "Geen"
df["document_referal"] = "Geen"

# Gebruik een sessie voor snellere requests
session = requests.Session()

def scrape_document_section(url, section_name):
    """Scrapet documentnummers uit een specifieke sectie en stopt als er geen nieuwe data meer is."""
    document_numbers = set()
    page = 1

    while True:
        paged_url = f"{url}?pu={page}#tabcontrol-1"

        try:
            time.sleep(1)

            response = session.get(paged_url, timeout=10)
            if response.status_code != 200:
                break  # Stop als de pagina niet laadt

            soup = BeautifulSoup(response.text, "html.parser")
            document_sections = soup.find_all("div", {"class": "mt-16", "data-e2e-name": "documents-section"})

            found_new_documents = False  # Flag om te checken of er nieuwe data is

            for section in document_sections:
                header = section.find("h2")
                if not header:
                    continue

                if section_name in header.text.strip():
                    document_rows = section.find_all("td")[0::4]  # Pakt documentnummers
                    doc_nums = {doc.text.strip() for doc in document_rows if doc.text.strip()}

                    if doc_nums - document_numbers:  # Check of er nieuwe documenten zijn
                        found_new_documents = True

                    document_numbers.update(doc_nums)

            if not found_new_documents:  # Stop als er geen nieuwe documenten zijn
                break

        except Exception as e:
            print(f"❌ Fout bij {paged_url}: {e}")
            break

        page += 1

    return ";".join(document_numbers) if document_numbers else "Geen"

try:
    # Loop door de gefilterde dataset en scrape de documentnummers
    for index, row in df.iterrows():
        url = row["document_sourceURL"]
        document_id = row["document_id"]

        try:
            df.at[index, "document_attachments"] = scrape_document_section(url, "Bijlagen bij dit e-mailbericht")
            df.at[index, "document_same_emailconvo"] = scrape_document_section(url, "Berichten uit dezelfde e-mailconversatie")
            df.at[index, "document_related"] = scrape_document_section(url, "Gerelateerde documenten")
            df.at[index, "document_referal"] = scrape_document_section(url, "Documenten die naar dit document verwijzen")

            print(f"✅ Gescraped document {document_id}")

        except Exception as e:
            print(f"❌ Fout bij document {document_id}: {e}")

        time.sleep(1)  # Delay om overbelasting te voorkomen

except KeyboardInterrupt:
    print("\n🛑 Scraping onderbroken! Opslaan van de huidige voortgang...")
finally:
    # Sla alleen de gescrapete subset op als een nieuw bestand
    df.to_csv("df_documents_vanaf_210-1626_tm_219-2695.csv", index=False)
    print("📁 Resultaten opgeslagen in df_documents_vanaf_205-3037_tm_205-3365.csv")

print("🎉 Scraping voltooid.")


✅ Gescraped document 210-1626
✅ Gescraped document 210-1627
✅ Gescraped document 210-1628
✅ Gescraped document 210-1629
✅ Gescraped document 210-1630
✅ Gescraped document 210-1631
✅ Gescraped document 210-1632
✅ Gescraped document 210-1633
✅ Gescraped document 210-1634
✅ Gescraped document 210-1635
✅ Gescraped document 210-1636
✅ Gescraped document 210-1637
✅ Gescraped document 210-1638
✅ Gescraped document 210-1639
✅ Gescraped document 210-1640
✅ Gescraped document 210-1641
✅ Gescraped document 210-1642
✅ Gescraped document 210-1643
✅ Gescraped document 210-1644
✅ Gescraped document 210-1645
✅ Gescraped document 210-1646
✅ Gescraped document 210-1647
✅ Gescraped document 210-1648
✅ Gescraped document 210-1649
✅ Gescraped document 210-1650
✅ Gescraped document 210-1651
✅ Gescraped document 210-1652
✅ Gescraped document 210-1653
✅ Gescraped document 210-1654
✅ Gescraped document 210-1655
✅ Gescraped document 210-1656
✅ Gescraped document 210-1657
✅ Gescraped document 210-1658
✅ Gescrape

In [ ]:
import pandas as pd
import re

# Inlezen van de CSV
input_file = "../../woogle/WoogleDumps/df_documents.csv"
df = pd.read_csv(input_file)

def extract_document_id_vws(title):
    # Als het een besluitbrief is, vul een placeholderwaarde in
    if title.lower() == "besluitbrief.pdf":
        return "N/A"

    # Match patronen
    match = re.search(r'VWS-WC-(\d{2,3}\.\d{2}|\d{2,3})-(\d+)', title)
    if match:
        return match.group(2)

    match = re.search(r'VWS-WC-\d{3}-([A-Za-z]*\d+[A-Za-z]*)', title)
    if match:
        return match.group(1)

    match = re.search(r'VWS-WOO-Woo-(\d+)', title)
    if match:
        return match.group(1)

    match = re.search(r'VWS-WOO-WOO-(\d+)', title)
    if match:
        return match.group(1)

    match = re.search(r'CCMO-WOO-\d{2}-(\d+)', title)
    if match:
        return match.group(1)

    match = re.search(r'VWS-WOO-Spc-(\d+)', title)
    if match:
        return match.group(1)

    match = re.search(r'VWS-WOO-Spc2-(\d+)', title)
    if match:
        return match.group(1)

    match = re.search(r'VWS-WOO-\d{2,3}[a-zA-Z]?-(\d+)', title)
    if match:
        return match.group(1)

    return None  # Als er geen match is

# Nieuwe kolom toevoegen
df["document_id_vws"] = df["document_title"].apply(extract_document_id_vws)

# Opslaan naar een nieuwe CSV
output_file = "df_documents_met_id.csv"
df.to_csv(output_file, index=False)

print(f"Verwerkt bestand opgeslagen als {output_file}")

In [ ]:
import pandas as pd

# Inlezen van de CSV met expliciet `document_id_vws` als string
input_file = "df_documents_met_id.csv"
df = pd.read_csv(input_file, dtype={"document_id_vws": str})

# Specifieke wijzigingen toepassen
df.loc[df["document_title"] == "VWS-WC-001-csmsberichtenmpsg-SMS_Berichten_MP_SG.pdf", "document_id_vws"] = "csmsberichtenmpsg"
df.loc[df["document_title"] == "VWS-WC-038-document-Bijlage_B_-_Openbaar_te_maken_overzicht_agenda-afspraken.pdf", "document_id_vws"] = "document"

# Opslaan naar een nieuwe CSV zonder extra quotes
output_file = "../../dataset/ALL/df_documents.csv"
df.to_csv(output_file, index=False)

print(f"Verwerkt bestand opgeslagen als {output_file}")
